In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Fri Jun  6 15:19:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.28                 Driver Version: 576.28         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   54C    P8             17W /   88W |     426MiB /   8192MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from datasets import load_dataset, Audio, concatenate_datasets, DatasetDict
import os

clips_dir = "D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/clips"
data_files = {
    "train": "D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/train.csv",
    "test": "D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/test.csv",
    "validation": "D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/dev.csv"
}

dataset = load_dataset("csv", data_files=data_files)


def add_full_path(example):
    example["path"] = os.path.join(clips_dir, example["path"])
    return example


dataset = dataset.map(add_full_path)

dataset = dataset.cast_column("path", Audio(sampling_rate=16000))

for split in dataset:
    dataset[split] = dataset[split].rename_columns({
        "path": "audio",
        "sentence": "text"
    })

for split in dataset:
    dataset[split] = dataset[split].remove_columns(
        [col for col in dataset[split].column_names if col not in ["audio", "text"]]
    )

train_dataset = concatenate_datasets([dataset["train"], dataset["validation"]])
test_dataset = dataset["test"]

common_voice = DatasetDict()

common_voice["train"] = train_dataset
common_voice["test"] = test_dataset

print(common_voice)

C:\anaconda3\envs\torch_gpu\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 40397
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 10668
    })
})


In [3]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [4]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Persian", task="transcribe")

In [5]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Persian", task="transcribe")

In [6]:
print(common_voice["train"][0])

{'audio': {'path': 'D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/clips\\common_voice_fa_24473237.mp3', 'array': array([-3.49245965e-10,  1.39698386e-09, -3.49245965e-10, ...,
        2.97557563e-07,  1.62050128e-07,  3.09199095e-07]), 'sampling_rate': 16000}, 'text': 'خشم کسی را دامن زدن'}


In [7]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [8]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [9]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"])

In [10]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [11]:
model.generation_config.language = "Persian"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [14]:
import evaluate

metric = evaluate.load("wer")

In [15]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [16]:
import numpy as np
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")

lengths = [len(tokenizer(sample["text"]).input_ids) for sample in dataset["test"]]
max_length = max(lengths)
percentile_95 = int(np.percentile(lengths, 95))

print("Max length:", max_length)
print("95th percentile:", percentile_95)

In [17]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-persian-v1",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    # group_by_length=True,
    learning_rate=4e-5,
    warmup_steps=500,
    # max_steps=4000,
    num_train_epochs=3,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    # eval_strategy="no",
    save_strategy="steps",
    save_total_limit=1,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=48,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=50,
    logging_dir="./logs",
    report_to=["tensorboard"],
    # load_best_model_at_end=False,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    resume_from_checkpoint='last-checkpoint',
)

In [18]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

C:\Users\A L I\AppData\Local\Temp\ipykernel_15804\3731876365.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [19]:
processor.save_pretrained(training_args.output_dir)

[]

In [ ]:
trainer.train()
# trainer.train(resume_from_checkpoint=True)

In [ ]:
# tensorboard --logdir=./logs --port=6006
